In [ ]:
# Kmeans 에러시 threadpoolctl 업그레이드 요망

# pip install threadpoolctl==3.1.0
# import threadpoolctl
# print(threadpoolctl.__version__)

In [16]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"


In [17]:
import os
os.chdir(r'E:\제출용데이터dir\천안시데이터')
# os.chdir(r'C:\Users\USER\Desktop\천안시데이터')


# os.chdir(r'C:\Users\USER\Desktop\데이터\천안시데이터 (2)')

# os.chdir('/Users/iseunghyeon/Desktop/프로젝트/data/제출용데이터dir/천안시데이터')


In [3]:
apiKey='1d49e941371e685a965fba06520cda68'
apiKey2='2Z4X+NCMmHm8pFVALLw9qdxwZN4RZOilmbir7hj1MoaKCFCm0TPJG0WT7eCqNm3ijXOkg30FgQF7IRr5d8rUQg=='


# 1. 데이터 전처리

> ## 1.1 주거정보 전처리

In [4]:
'''
2023 7월 "주소DB" 에서 충남의 데이터만 가져옵니다.
지번_충청남도.txt

txt파일을 데이터 프레임으로 불러오기 위해, '|' 구분자 기호를 통해서, 칼럼을 분리하고, 

충남에서, 천안시데이터만 추출합니다.
'''

addressData = pd.read_csv('./지번_충청남도.txt',encoding='cp949',sep='|',header=None)
col_list= ['관리번호','일련번호','법정동코드','시도명','시군구명','법정읍면동명','법정리명','산여부','번지','호','대표여부']
addressData.columns=col_list
addressData = addressData[addressData['시군구명'].isin(['천안시 동남구','천안시 서북구'])]
addressData

'\n2023 7월 "주소DB" 에서 충남의 데이터만 가져옵니다.\n지번_충청남도.txt\n\ntxt파일을 데이터 프레임으로 불러오기 위해, \'|\' 구분자 기호를 통해서, 칼럼을 분리하고, \n\n충남에서, 천안시데이터만 추출합니다.\n'

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부
0,4413025629101030004000001,1,4413325629,충청남도,천안시 서북구,직산읍,모시리,0,103,4,1
1,4413025632102760000000001,1,4413325632,충청남도,천안시 서북구,직산읍,신갈리,0,276,0,1
2,4413034027103000000000001,1,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,0,1
3,4413034027103000000000001,3,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,3,0
4,4413034027103000000000001,5,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,5,0
...,...,...,...,...,...,...,...,...,...,...,...
93506,4413331038102740103012993,5,4413331038,충청남도,천안시 서북구,입장면,독정리,0,274,3,0
93507,4413331038102740103012993,6,4413331038,충청남도,천안시 서북구,입장면,독정리,0,274,4,0
93508,4413331038102840000013127,1,4413331038,충청남도,천안시 서북구,입장면,독정리,0,284,0,1
93509,4413331038102870000015693,1,4413331038,충청남도,천안시 서북구,입장면,독정리,0,287,0,1


> ### 1.1.1 카카오 api를 통한 해당 지번건물의 상세한 정보 얻기

주소 DB의 시군구명, 법정읍면동명, 번지, 호 를 조합하여, 카카오 api를 통해 도로명 주소와, 해당 위도,경도 좌표
를 얻습니다.

In [31]:
cnt=0

addressCopy = addressData.copy()
addressCopy = addressCopy[:100]
headers = {'Authorization': 'KakaoAK {}'.format(apiKey) }

addressCopy['byAPI'] =None
for idx in tqdm(range(len(addressCopy))):
  try:
    address1 = addressCopy.loc[idx,'시군구명']
    address2 = addressCopy.loc[idx,'법정읍면동명']
    address3 = addressCopy.loc[idx,'번지']
    address4 = addressCopy.loc[idx,'호']
    address =  address1 + ' ' + address2 + ' ' + str(address3) + ' ' + str(address4)

    searching = address
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(searching)
    data = requests.post(url,headers=headers)
    result = json.loads(data.text)
    # print(result)
    documents = result['documents']
    

    addressCopy.at[idx,'byAPI'] = documents[0]
  except:
    cnt+=1
    print('쿼리값을 못찾는중:',cnt)
    pass

addressCopy

100%|██████████| 100/100 [00:05<00:00, 19.21it/s]


,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,byAPI
0,4413025629101030004000001,1,4413325629,충청남도,천안시 서북구,직산읍,모시리,0,103,4,1,{'address': {'address_name': '충남 천안시 서북구 직산읍 모...
1,4413025632102760000000001,1,4413325632,충청남도,천안시 서북구,직산읍,신갈리,0,276,0,1,{'address': {'address_name': '충남 천안시 서북구 직산읍 군...
2,4413034027103000000000001,1,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,0,1,{'address': {'address_name': '충남 천안시 동남구 성남면 대...
3,4413034027103000000000001,3,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,3,0,{'address': {'address_name': '충남 천안시 동남구 성남면 신...
4,4413034027103000000000001,5,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,5,0,{'address': {'address_name': '충남 천안시 동남구 성남면 신...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,4413110100100210002015782,2,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,21,3,0,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...
96,4413110100100210004014017,1,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,21,4,1,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...
97,4413110100100210005015975,1,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,21,5,1,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...
98,4413110100100210005015975,2,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,21,1,0,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...


In [22]:
'''
카카오 api를 통해
도로명주소와, 위도, 경도 좌표를 얻을 수 있게된다.
'''
addressCopy.loc[2,'byAPI']

'\n카카오 api를 통해\n도로명주소와, 위도, 경도 좌표를 얻을 수 있게된다.\n'

{'address': {'address_name': '충남 천안시 동남구 성남면 대정리 300',
  'b_code': '4413134024',
  'h_code': '4413134000',
  'main_address_no': '300',
  'mountain_yn': 'N',
  'region_1depth_name': '충남',
  'region_2depth_name': '천안시 동남구',
  'region_3depth_h_name': '성남면',
  'region_3depth_name': '성남면 대정리',
  'sub_address_no': '',
  'x': '127.204418833386',
  'y': '36.738012181472'},
 'address_name': '충남 천안시 동남구 성남면 대정리 300',
 'address_type': 'REGION_ADDR',
 'road_address': None,
 'x': '127.204418833386',
 'y': '36.738012181472'}

> ### 1.1.2 아파트, 빌라 데이터 추출하기

카카오 api에서 제공되는 데이터인 건물에 대한 빌딩이름으로, 사람들이 거주하는 건물을 파악합니다.

building_name의 값이 '아파트' 또는 '빌라' 라는 단어가 포함하는 데이터를 파악합니다.

In [32]:
# 카카오 api를 통해 추출한 값을 담은 byApi칼럼 값들에서 road_address 의 building_name을 통해 ,건물명을 추출합니다.
addressCopy['건물명'] = None
for idx in tqdm(range(len(addressCopy))):
    try:    
        data = addressCopy.at[idx,'byAPI'].get('road_address',None).get('building_name',None)
        if len(data) !=0: 
           addressCopy.at[idx,'건물명'] = data
    except:
        pass
addressCopy.head()

100%|██████████| 100/100 [00:00<00:00, 33341.05it/s]


,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,byAPI,건물명
0,4413025629101030004000001,1,4413325629,충청남도,천안시 서북구,직산읍,모시리,0,103,4,1,{'address': {'address_name': '충남 천안시 서북구 직산읍 모...,None
1,4413025632102760000000001,1,4413325632,충청남도,천안시 서북구,직산읍,신갈리,0,276,0,1,{'address': {'address_name': '충남 천안시 서북구 직산읍 군...,(주)두호스테인레스
2,4413034027103000000000001,1,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,0,1,{'address': {'address_name': '충남 천안시 동남구 성남면 대...,None
3,4413034027103000000000001,3,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,3,0,{'address': {'address_name': '충남 천안시 동남구 성남면 신...,None
4,4413034027103000000000001,5,4413134027,충청남도,천안시 동남구,성남면,신사리,0,300,5,0,{'address': {'address_name': '충남 천안시 동남구 성남면 신...,None


In [33]:
# 건물명 데이터 셋 저장
# addressCopy.to_pickle('./addressByAPI_all_building_0816.pickle')

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,byAPI,건물명


In [34]:
# 건물명 데이터 셋 불러오기
all_building = pd.read_pickle('./addressByAPI_all_building_0816.pickle')
all_building.head(2)

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,byAPI,건물명
0,4413025629101030004000001,1,4413325629,충청남도,천안시 서북구,직산읍,모시리,0,103,4,1,{'address': {'address_name': '충남 천안시 서북구 직산읍 모...,NA
1,4413025632102760000000001,1,4413325632,충청남도,천안시 서북구,직산읍,신갈리,0,276,0,1,{'address': {'address_name': '충남 천안시 서북구 직산읍 군...,(주)두호스테인레스


In [40]:
# 건물명 데이터 셋을 불러와서, 건물이름에 "아파트" 와 "빌라"가 포함되어 있는 데이터만 추출 합니다.
apt_villa = all_building[all_building['건물명'].str.contains('아파트|빌라', case=False)]
apt_villa.head(2)

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,byAPI,건물명
132,4413110100100430000014153,1,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,43,0,1,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...,천안맨션아파트
489,4413110100101650000019452,1,4413110100,충청남도,천안시 동남구,대흥동,NaN,0,165,0,1,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...,미도아파트


> ### 1.1.3 공동주택 단지목록 코드 데이터와 국토교통부_공동주택 기본 정보제공 서비스 api를 통한 주거지역 정보 확보

주거지역에 대한 정보를 더 확보하기 위하여, 해당 데이터들을 사용하여 주거지역정보를 더 확보합니다

In [41]:
apt_xl = pd.read_excel('./공동주택_단지목록_Code_20221129.xlsx')

'''
광역시도 칼럼의 "세종특별자치시" 의 시군구 칼럼의 값이 NaN값이기 때문에
NaN값은 삭제합니다.
'''

apt_xl.dropna(subset=['시군구'],inplace=True)

'''
이후 시군구칼럼의 값에, 천안시가 포함되어 있는 데이터만 추출합니다.
''' 
apt_xl = apt_xl[apt_xl['시군구'].str.contains('천안')]
apt_xl.head(2)


'\n광역시도 칼럼의 "세종특별자치시" 의 시군구 칼럼의 값이 NaN값이기 때문에\nNaN값은 삭제합니다.\n'

'\n이후 시군구칼럼의 값에, 천안시가 포함되어 있는 데이터만 추출합니다.\n'

,No,광역시도,시군구,동읍면,리,건물명,법정동코드,단지코드
17863,17864,충청남도,천안동남구,구성동,NaN,구성 현대I-PARK 아파트,4413110800,A33006001
17864,17865,충청남도,천안동남구,구성동,NaN,구성동 쌍용아파트,4413110800,A33006002


In [42]:
# 주소 DB에서 천안시의 아파트, 빌라만 추출한 데이터에 (apt_villa 데이터)
# 국토교통부_공동주택 기본 정보 제공 서비스의 데이터를 보충하이 위하여
# 2개의 '법정동코드'를 비교하여 apt_villa 에 포함하지 않은 데이터만 추가시킵니다.
apt_villa_code = list(apt_villa['법정동코드'].values)

apt_xl_code = list(apt_xl['법정동코드'].values)


apt_xl_code_toApi = [elm for elm in apt_xl_code if elm not in apt_villa_code]
apt_xl_toApi =apt_xl[apt_xl['법정동코드'].isin(apt_xl_code_toApi)]
apt_xl_toApi.head(2)


,No,광역시도,시군구,동읍면,리,건물명,법정동코드,단지코드
17877,17878,충청남도,천안동남구,목천읍,서리,재동보라,4413125021,A33074102
17883,17884,충청남도,천안동남구,목천읍,운전리,연합초원아파트,4413125029,A10025557


In [43]:
# 예시를 위해서 5개의 데이터만 넣습니다.
apt_xl_toApi = apt_xl_toApi.head(2)

import requests
import xml.etree.ElementTree as ET
# apiKey2='2Z4X+NCMmHm8pFVALLw9qdxwZN4RZOilmbir7hj1MoaKCFCm0TPJG0WT7eCqNm3ijXOkg30FgQF7IRr5d8rUQg=='
url = 'http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo'

apt_xl_toApi.reset_index(inplace=True,drop=True)

apt_xl_toApi['byApi2'] = None
for idx in tqdm(range(len(apt_xl_toApi))):
    kaptCode = apt_xl_toApi.at[idx,'단지코드']

    params ={'serviceKey' : apiKey2, 'kaptCode' : kaptCode } 

    response = requests.get(url, params=params)
    api = response.text
    root = ET.fromstring(api)
    data_dict = {}
    for item in root.findall(".//item/*"):
        data_dict[item.tag] = item.text

    apt_xl_toApi.at[idx,'byApi2'] = data_dict

apt_xl_toApi


100%|██████████| 2/2 [00:00<00:00,  7.73it/s]


,No,광역시도,시군구,동읍면,리,건물명,법정동코드,단지코드,byApi2
0,17878,충청남도,천안동남구,목천읍,서리,재동보라,4413125021,A33074102,"{'bjdCode': '4413125021', 'codeAptNm': '아파트', ..."
1,17884,충청남도,천안동남구,목천읍,운전리,연합초원아파트,4413125029,A10025557,"{'bjdCode': '4413125029', 'codeAptNm': '아파트', ..."


In [44]:
# api를 통해서 불러온 데이터들중 kaptAddr 의 값은 도로명 주소를 나타내므로, 
# 주소칼럼을 만들어, kaptAddr의 값을 할당합니다.
# 이 '주소'값은 추후 카카오 api를 통해 데이터를 통일하는 과정을 거칩니다.
apt_xl_toApi['주소'] = apt_xl_toApi['byApi2'].apply(lambda elm:elm.get('kaptAddr'))
apt_xl_toApi.head(2)

,No,광역시도,시군구,동읍면,리,건물명,법정동코드,단지코드,byApi2,주소
0,17878,충청남도,천안동남구,목천읍,서리,재동보라,4413125021,A33074102,"{'bjdCode': '4413125021', 'codeAptNm': '아파트', ...",충청남도 천안동남구 목천읍 서리 58-1 재동보라
1,17884,충청남도,천안동남구,목천읍,운전리,연합초원아파트,4413125029,A10025557,"{'bjdCode': '4413125029', 'codeAptNm': '아파트', ...",충청남도 천안동남구 목천읍 운전리 380-3 연합초원아파트


In [59]:
# 카카오 api를 통하여, 건물명에 대한 정보를 얻습니다.
apt_xl_toApiByKakao = address2api(apt_xl_toApi,apiKey)
apt_xl_toApiByKakao.head(2)


100%|██████████| 2/2 [00:00<00:00,  9.09it/s]


,No,광역시도,시군구,동읍면,리,건물명,법정동코드,단지코드,byApi2,주소,byAPI
0,17878,충청남도,천안동남구,목천읍,서리,재동보라,4413125021,A33074102,"{'bjdCode': '4413125021', 'codeAptNm': '아파트', ...",충청남도 천안동남구 목천읍 서리 58-1 재동보라,{'address': {'address_name': '충남 천안시 동남구 목천읍 서...
1,17884,충청남도,천안동남구,목천읍,운전리,연합초원아파트,4413125029,A10025557,"{'bjdCode': '4413125029', 'codeAptNm': '아파트', ...",충청남도 천안동남구 목천읍 운전리 380-3 연합초원아파트,{'address': {'address_name': '충남 천안시 동남구 목천읍 운...


In [60]:
apt_xl_toApiByKakao_will_merge = apt_xl_toApiByKakao[['법정동코드','byAPI']]


# 천안시의 거주지역 데이터를 모두 합한 데이터 셋인 concat_df를 만듭니다.
concat_df = pd.merge(apt_villa,apt_xl_toApiByKakao_will_merge,on=['법정동코드'],how='outer')
concat_df['byAPI'] = concat_df['byAPI_x'].fillna(concat_df['byAPI_y'])
concat_df.drop(['byAPI_x','byAPI_y'],axis=1,inplace=True)
concat_df.head()

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,건물명,byAPI
0,4413110100100430000014153,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,43.0,0.0,1.0,천안맨션아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...
1,4413110100101650000019452,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,165.0,0.0,1.0,미도아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...
2,4413110100102140000000001,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,214.0,0.0,1.0,빌라,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...
3,4413110100102370000022867,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,237.0,0.0,1.0,반석빌라,{'address': {'address_name': '충남 천안시 동남구 대흥동 2...
4,4413110200100010001011579,1.0,4413110200,충청남도,천안시 동남구,성황동,NaN,0.0,1.0,24.0,1.0,태양빌라,{'address': {'address_name': '충남 천안시 동남구 성황동 1...


In [61]:
len(apt_villa)+len(apt_xl_toApiByKakao_will_merge) == len(concat_df)


True

> ### 1.1.4 공동주택 데이터 전처리

https://www.data.go.kr/data/15111700/fileData.do


공공 데이터 포털의 "충청남도 천안시_공동주택 현황" 데이터를 통해, 

부족한 아파트와 빌라 정보를 보충합니다.

"충청남도 천안시_공동주택 현황" 의 공동주택명은 카카오api 에서 명시한 건물이름과 동일 하지 않으므로, 주소칼럼을 통해, 카카오api를 활용하여 동일한 건물명을 갖도록 합니다

In [62]:
공동주택 = pd.read_csv('./충청남도 천안시_공동주택 현황_20221230.csv',encoding='cp949')
공동주택.head()

,지역,공동주택명,주소,세대수,층수,동수,준공연도,관리실번호
0,천안시,한성스위트빌아파트(1차),충청남도 천안시 서북구 부성6길 11(두정동),600,15,6,2002-08-26,041-554-1133
1,천안시,계룡리슈빌아파트,충청남도 천안시 서북구 부성8길 29(두정동),386,15,4,2004-03-18,041-557-1472
2,천안시,두정경남아너스빌,충청남도 천안시 서북구 두정중11길 17(두정동),458,15,6,2004-06-10,041-556-3307
3,천안시,두정e편한세상(1차),충청남도 천안시 서북구 두정중2길 12(두정동),466,15,9,2004-06-25,041-561-8040
4,천안시,두정푸르지오아파트(3차),충청남도 천안시 서북구 성정두정로 142(두정동),326,15,4,2003-09-09,041-554-9116


In [63]:
# 카카오 api 를 활용해서, 도로명 주소를 통해 -> 건물에 대한 상세 주소 알기
def address2api(DF,apiKey) -> pd.DataFrame :
  cnt=0
  addressCopy = DF.copy()
  headers = {'Authorization': 'KakaoAK {}'.format(apiKey) }
  addressCopy['byAPI'] =None
  for idx in tqdm(range(len(addressCopy))):
    try:

      address = addressCopy.at[idx,'주소']

      searching = address
      url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(searching)
      data = requests.post(url,headers=headers)
      result = json.loads(data.text)
      # print(result)
      documents = result['documents']
      addressCopy.at[idx,'byAPI'] = documents[0]
    except:
      cnt+=1
      if cnt%50==0:
        print('쿼리값을 못찾는중:',cnt) 
      pass

  return addressCopy


# 도로명주소 -> API값 얻기

공동주택 = 공동주택.head(10) # 예시를 위해서 5개의 데이터만 넣습니다.
공동주택byApi = address2api(공동주택,apiKey)
공동주택byApi.head()


100%|██████████| 10/10 [00:00<00:00, 18.12it/s]


,지역,공동주택명,주소,세대수,층수,동수,준공연도,관리실번호,byAPI
0,천안시,한성스위트빌아파트(1차),충청남도 천안시 서북구 부성6길 11(두정동),600,15,6,2002-08-26,041-554-1133,{'address': {'address_name': '충남 천안시 서북구 두정동 1...
1,천안시,계룡리슈빌아파트,충청남도 천안시 서북구 부성8길 29(두정동),386,15,4,2004-03-18,041-557-1472,{'address': {'address_name': '충남 천안시 서북구 두정동 1...
2,천안시,두정경남아너스빌,충청남도 천안시 서북구 두정중11길 17(두정동),458,15,6,2004-06-10,041-556-3307,{'address': {'address_name': '충남 천안시 서북구 두정동 1...
3,천안시,두정e편한세상(1차),충청남도 천안시 서북구 두정중2길 12(두정동),466,15,9,2004-06-25,041-561-8040,{'address': {'address_name': '충남 천안시 서북구 두정동 1...
4,천안시,두정푸르지오아파트(3차),충청남도 천안시 서북구 성정두정로 142(두정동),326,15,4,2003-09-09,041-554-9116,{'address': {'address_name': '충남 천안시 서북구 두정동 1...


In [68]:
m1 = concat_df.copy()
m2 = 공동주택byApi.copy()

def get_address_name(row):
    if row['byAPI'] is None:
        return None
    return row['byAPI'].get('road_address',{}).get('address_name',None)

m1['address_name'] = m1.apply(get_address_name,axis=1)
m2['address_name'] = m1.apply(get_address_name,axis=1)


m2_2 = m2[['byAPI','address_name']]
m2_2.head(2)

m1_address_name_list = list(m1['address_name'].values)
m2_address_name_list = list(m2['address_name'].values)
len(m1_address_name_list),len(m2_address_name_list)

set_m1_address_name_list = set(m1_address_name_list)
set_m2_address_name_list = set(m2_address_name_list)
inter_list = set_m1_address_name_list.intersection(set_m2_address_name_list)
len(inter_list)

# outer 조인을 통하여, 충청남도 천안시_공동주택 현황_20221230.csv 파일과의 데이터와 결합을 한다.
m_merge = pd.merge(m1,m2_2,on=['address_name'],how='outer')
m_merge.head(2)
m_merge['byAPI'] = m_merge['byAPI_x'].fillna(m_merge['byAPI_y'])
m_merge.drop(['byAPI_x','byAPI_y'],axis=1,inplace=True)
m_merge.head(2)
len(m_merge)

,byAPI,address_name
0,{'address': {'address_name': '충남 천안시 서북구 두정동 1...,충남 천안시 동남구 대흥로 267
1,{'address': {'address_name': '충남 천안시 서북구 두정동 1...,충남 천안시 동남구 옛시청길 6


(1167, 10)

9

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,건물명,byAPI_x,address_name,byAPI_y
0,4413110100100430000014153,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,43.0,0.0,1.0,천안맨션아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...,충남 천안시 동남구 대흥로 267,{'address': {'address_name': '충남 천안시 서북구 두정동 1...
1,4413110100101650000019452,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,165.0,0.0,1.0,미도아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...,충남 천안시 동남구 옛시청길 6,{'address': {'address_name': '충남 천안시 서북구 두정동 1...


,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,건물명,address_name,byAPI
0,4413110100100430000014153,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,43.0,0.0,1.0,천안맨션아파트,충남 천안시 동남구 대흥로 267,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...
1,4413110100101650000019452,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,165.0,0.0,1.0,미도아파트,충남 천안시 동남구 옛시청길 6,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...


1169

In [69]:
m_merge_uq = m_merge.drop_duplicates(subset='address_name',keep='first')
len(m_merge_uq)
m_merge_uq.head(2)

696

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,번지,호,대표여부,건물명,address_name,byAPI
0,4413110100100430000014153,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,43.0,0.0,1.0,천안맨션아파트,충남 천안시 동남구 대흥로 267,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...
1,4413110100101650000019452,1.0,4413110100,충청남도,천안시 동남구,대흥동,NaN,0.0,165.0,0.0,1.0,미도아파트,충남 천안시 동남구 옛시청길 6,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...


In [ ]:
# 건물명 채우기
def fill_building_name(row):
    if pd.isna(row['건물명']):
        return row['byAPI'].get('road_address',{}).get('building_name',None)
    return row['건물명']
m_merge_uq['건물명'] = m_merge_uq.apply(fill_building_name,axis=1)


In [ ]:
# m_merge_uq.to_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/주거단지/addressByAPIpickle08191010.pickle')

> ## 1.2 제조업체 전처리

천안시제조업체데이터 데이터를 엑셀에서, 사용하지 않은 칼럼은 제거후
천안시제조업체데이터_가공 엑셀 파일로 만들어서 전처리를 진행하였습니다.

In [70]:
facility = pd.read_excel('./천안시제조업체데이터_가공.xlsx')

facility


,시군\n기호,업종\n코드,기업체명,소 재 지
0,1.0,22.0,(주)동아수지산업,충청남도 천안시 서북구 직산읍 금곡로 52-5
1,1.0,33.0,KY컴퍼니,충청남도 천안시 서북구 직산읍 금곡로 52-20
2,1.0,29.0,N&P시스템,충청남도 천안시 서북구 직산읍 금곡로 52-11
3,1.0,33.0,RP이엔지,충청남도 천안시 서북구 직산읍 금곡로 57
4,1.0,22.0,남주테크,충청남도 천안시 서북구 직산읍 금곡로 45
...,...,...,...,...
3186,1.0,23.0,휴먼이엔티주식회사,충청남도 천안시 서북구 성거읍 망향로 903-6
3187,1.0,10.0,흥림농산,충청남도 천안시 동남구 수신면 장산동길 168-27
3188,1.0,18.0,희성폴리머(주),충청남도 천안시 서북구 성환읍 천안대로 2131 (성환읍) 외 1필지
3189,1.0,29.0,희영,충청남도 천안시 서북구 직산읍 금곡로 141 ((주)그린테크산업) (총 3 필지) ...


> ### 1.2.1 제조업체 데이터의 위도,경도 좌표를 얻기


소재지 칼럼의 도로명 주소를 이용해 위도,경도 좌표를 얻기위해 카카오 api를 이용해 정보를 얻습니다 

    [결측치보충]

    카카오 api를 통해 추출하지 못한 정보는, 웹 서핑을 통해 결측치를 보충하였습니다.

In [71]:
facility_cp = facility.copy()
facility_cp = facility_cp.head()
facility_cp = facility_cp.rename(columns={'소  재  지':'주소'})
facility_cp_byApi = address2api(facility_cp,apiKey)
facility_cp_byApi

,시군\n기호,업종\n코드,기업체명,주소
0,1.0,22.0,(주)동아수지산업,충청남도 천안시 서북구 직산읍 금곡로 52-5
1,1.0,33.0,KY컴퍼니,충청남도 천안시 서북구 직산읍 금곡로 52-20
2,1.0,29.0,N&P시스템,충청남도 천안시 서북구 직산읍 금곡로 52-11
3,1.0,33.0,RP이엔지,충청남도 천안시 서북구 직산읍 금곡로 57
4,1.0,22.0,남주테크,충청남도 천안시 서북구 직산읍 금곡로 45


100%|██████████| 5/5 [00:00<00:00, 18.48it/s]


,시군\n기호,업종\n코드,기업체명,주소,byAPI
0,1.0,22.0,(주)동아수지산업,충청남도 천안시 서북구 직산읍 금곡로 52-5,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...
1,1.0,33.0,KY컴퍼니,충청남도 천안시 서북구 직산읍 금곡로 52-20,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...
2,1.0,29.0,N&P시스템,충청남도 천안시 서북구 직산읍 금곡로 52-11,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...
3,1.0,33.0,RP이엔지,충청남도 천안시 서북구 직산읍 금곡로 57,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...
4,1.0,22.0,남주테크,충청남도 천안시 서북구 직산읍 금곡로 45,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...


In [ ]:
# 제조업체 데이터 셋 facility_cp_byApi 을 저장합니다. 
# facility_cp_byApi.to_pickle('./제조업체데이터0819.pickle')

> ## 1.3 산업단지 전처리

In [72]:
industrial = pd.read_csv('./천안시산업단지데이터.csv',encoding='cp949')
industrial.head()

,단지명,회사명,공장대표주소(도로명),전화번호,데이터 기준 일자
0,천안산업기술일반산업단지,주식회사 두코,"충청남도 천안시 서북구 직산읍 직산로 136, 생산관 2105호 (직산읍)",1644-3880,2022-06-23
1,천안마정일반산업단지,(주)케이와이컴퍼니,충청남도 천안시 서북구 직산읍 금곡로 52-20,1599-3176,2022-06-23
2,천안산업기술일반산업단지,주식회사 바이콘,"충청남도 천안시 서북구 직산읍 직산로 136, 생산관 2305호",1566-7185,2022-06-23
3,천안산업기술일반산업단지,(주)인피닉스,"충청남도 천안시 서북구 직산읍 직산로 136, 충남테크노파크 영상관 506호",1688-0201,2022-06-23
4,천안산업기술일반산업단지,(주)에스피에스,"충청남도 천안시 서북구 직산읍 직산로 136, 3102호(충남테크노파크 생산관)",070-8871-5835,2022-06-23


In [73]:
industrial_cp = industrial.copy()
industrial_cp = industrial_cp.head()
industrial_cp = industrial_cp.rename(columns={'공장대표주소(도로명)':'주소'})
industrial_cp
industrial_cp_byApi = address2api(industrial_cp,apiKey)
industrial_cp_byApi

,단지명,회사명,주소,전화번호,데이터 기준 일자
0,천안산업기술일반산업단지,주식회사 두코,"충청남도 천안시 서북구 직산읍 직산로 136, 생산관 2105호 (직산읍)",1644-3880,2022-06-23
1,천안마정일반산업단지,(주)케이와이컴퍼니,충청남도 천안시 서북구 직산읍 금곡로 52-20,1599-3176,2022-06-23
2,천안산업기술일반산업단지,주식회사 바이콘,"충청남도 천안시 서북구 직산읍 직산로 136, 생산관 2305호",1566-7185,2022-06-23
3,천안산업기술일반산업단지,(주)인피닉스,"충청남도 천안시 서북구 직산읍 직산로 136, 충남테크노파크 영상관 506호",1688-0201,2022-06-23
4,천안산업기술일반산업단지,(주)에스피에스,"충청남도 천안시 서북구 직산읍 직산로 136, 3102호(충남테크노파크 생산관)",070-8871-5835,2022-06-23


100%|██████████| 5/5 [00:00<00:00, 16.98it/s]


,단지명,회사명,주소,전화번호,데이터 기준 일자,byAPI
0,천안산업기술일반산업단지,주식회사 두코,"충청남도 천안시 서북구 직산읍 직산로 136, 생산관 2105호 (직산읍)",1644-3880,2022-06-23,{'address': {'address_name': '충남 천안시 서북구 직산읍 삼...
1,천안마정일반산업단지,(주)케이와이컴퍼니,충청남도 천안시 서북구 직산읍 금곡로 52-20,1599-3176,2022-06-23,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...
2,천안산업기술일반산업단지,주식회사 바이콘,"충청남도 천안시 서북구 직산읍 직산로 136, 생산관 2305호",1566-7185,2022-06-23,{'address': {'address_name': '충남 천안시 서북구 직산읍 삼...
3,천안산업기술일반산업단지,(주)인피닉스,"충청남도 천안시 서북구 직산읍 직산로 136, 충남테크노파크 영상관 506호",1688-0201,2022-06-23,{'address': {'address_name': '충남 천안시 서북구 직산읍 삼...
4,천안산업기술일반산업단지,(주)에스피에스,"충청남도 천안시 서북구 직산읍 직산로 136, 3102호(충남테크노파크 생산관)",070-8871-5835,2022-06-23,{'address': {'address_name': '충남 천안시 서북구 직산읍 삼...


In [96]:
# 산업단지 데이터 셋인 industrial_cp_byApi 을 저장합니다
# industrial_cp_byApi.to_pickle('./산업단지byAPI_0817.pickle')

# 2. 버스 정류장 좌표를 통해 주변 시설 구하기

>## 2.1데이터 불러오기

In [74]:
bus = pd.read_csv('./2022년_전국버스정류장 위치정보_데이터.csv')
bus.head(2)

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산


>## 2.2 카카오 api를 통하여, 천안시 버스정류장을 필터링하기
천안시의 버스정류장을 고려하기 위해서, 천안시의 버스정류장 데이터를 필터링한다.

제공된 버스정류장의 데이터에는 위도, 경도 좌표에 부여된 도시명과 맞지 않는 부분이 있기 때문에,

카카오 api를 활용하여, 도로명 주소를 구하고 도로명주소를 활용하여 버스 정류장이 천안시에만 있는 데이터만을 추출한다.


In [77]:
addressCopy = bus.copy()
addressCopy = addressCopy.head(10)
headers = {'Authorization': 'KakaoAK {}'.format(apiKey) }

addressCopy['coord2address'] = None

cnt=0 
for idx in tqdm(range(len(addressCopy))):
  try:

    x = addressCopy.at[idx,'경도']
    y =  addressCopy.at[idx,'위도']
    searching ='x={x}&y={y}'.format(x=str(x),y=str(y))
    url='https://dapi.kakao.com/v2/local/geo/coord2address.json?{}'.format(searching)
    data = requests.post(url,headers=headers)
    result = json.loads(data.text)
    documents = result['documents']
    
    addressCopy.at[idx,'coord2address'] = documents[0]
  except:
    cnt+=1
    if cnt%50==0:
      print('쿼리값을 못찾는중:',cnt) 

    pass

addressCopy.head()

100%|██████████| 10/10 [00:00<00:00, 17.65it/s]


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,coord2address
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
2,ASB214015456,오성슈퍼,36.951671,127.052784,20221012050123,15456.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
3,ASB214015459,팽성초교입구,36.952036,127.053031,20221012050123,15459.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."
4,ASB214015461,대사리입구,36.944598,127.053092,20221012050123,15461.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na..."


In [79]:
# 제공된 버스정류장의 데이터에선 정류장번호가 ASB214010001 인 데이터의 도시명은 아산시라고 되어있지만, 카카오 api를 통해 나온 도로명 주소는 평택시로 되어있다.
# 구글 맵을 통해, 위도 경도 좌표를 넣어보면, 구글 맵또한 평택시라고 명시되어 있다.
addressCopy.loc[0,'coord2address']

# '시' 를 구분하는 region_2depth_name 데이터를 통해, 제공된 버스 정류장 데이터에 새로운 칼럼을 부여하여,
# region_2depth_name 칼럼을 통해, 천안시인 데이터만을 필터링한다.
addressCopy['region_2depth_name'] = addressCopy['coord2address'].apply(lambda elm : elm.get('address').get('region_2depth_name'))
addressCopy.head()

{'road_address': None,
 'address': {'address_name': '경기 평택시 팽성읍 석근리 191-1',
  'region_1depth_name': '경기',
  'region_2depth_name': '평택시',
  'region_3depth_name': '팽성읍 석근리',
  'mountain_yn': 'N',
  'main_address_no': '191',
  'sub_address_no': '1',
  'zip_code': ''}}

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,coord2address,region_2depth_name
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
2,ASB214015456,오성슈퍼,36.951671,127.052784,20221012050123,15456.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
3,ASB214015459,팽성초교입구,36.952036,127.053031,20221012050123,15459.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시
4,ASB214015461,대사리입구,36.944598,127.053092,20221012050123,15461.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",평택시


In [80]:
# 천안시의 버스정류장만을 고려하기 위해, 버스정류장이 천안시에 위치해 있는 곳만 필터링하여 저장한다.
filtered_addressCopy = addressCopy[addressCopy['region_2depth_name']=='천안시']

# 전처리 데이터 저장
# filtered_addressCopy.to_pickle('./bus_coord2address.pickle')

>## 2.3 카카오 api를 통해 얻은 도로명 주소를 통해 반경 0.5km 이내에 있는 시설 종합하기

In [ ]:
# ToDo. 시,구, 읍/면/동, 리 뽑는 코드
# bus_station_Main.drop(columns='주변시설의총합',inplace=True)


In [82]:
# bus_coord2address : 천안시 버스정류장 파일
bus_station_Main = pd.read_pickle('./bus_coord2address.pickle') 
bus_station_Main.head(3)

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,coord2address,region_2depth_name
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",천안시 서북구
1,ASB285000078,송덕,36.923783,127.118976,20221012050123,78.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",천안시 서북구
2,ASB285000079,국립축산과학원,36.925931,127.111136,20221012050123,79.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",천안시 서북구


In [83]:
bus_station_cp = bus_station_Main.copy()
CateGroupCode = ['MT1','CS2','PS3','SC4','AC5','PK6','OL7','SW8','BK9','CT1','AG2','PO3','AT4','AD5','FD6','CE7','HP8','PM9']
code_df = pd.DataFrame(columns=CateGroupCode)
bus_code_df = pd.concat([bus_station_Main,code_df],axis=1)
bus_code_df.head(2)

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,coord2address,...,BK9,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ASB285000078,송덕,36.923783,127.118976,20221012050123,78.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
cnt=0
# 적용할 데이터프레임
# bus_code_df = 데이터프레임.copy()


# 예시 데이터 셋을 위해 데이터 2개만 정보를 추출합니다.!
bus_code_df = bus_code_df.head(2)


headers = {'Authorization': 'KakaoAK {}'.format(apiKey) }
for code in tqdm(CateGroupCode):
  for idx in tqdm(range(len(bus_code_df))):
    x = bus_code_df.at[idx,'경도']
    y = bus_code_df.at[idx,'위도']
    try:
      searching ='category_group_code={code}&x={x}&y={y}&radius={radius}'.format(code=code,x=str(x),y=str(y),radius=500)
      url='https://dapi.kakao.com/v2/local/search/category.json?{}'.format(searching)
      data=requests.post(url,headers=headers)
      result = json.loads(data.text)
      print(result)    
      
      

      bus_code_df.at[idx,code] = result.get('meta').get('total_count')


    except:
      # cnt+1
      # if cnt%50 ==0:
      #   print('안긁히고 있는 데이터 갯수:',cnt)
      pass
  # print(code,'코드 완료')
bus_code_df

100%|██████████| 18/18 [00:00<00:00, 50.28it/s]


,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명,coord2address,...,BK9,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9
0,ASB285000077,성환8리,36.921468,127.122774,20221012050123,77.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ASB285000078,송덕,36.923783,127.118976,20221012050123,78.0,34040,아산시,아산,"{'road_address': None, 'address': {'address_na...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 추출한 데이터 셋 bus_code_df 를 "Bus_around_500_count_08231838" 으로 저장합니다.
# bus_code_df.to_pickle('./Bus_around_500_count_08231838.pickle')

In [84]:
# Bus_around_500_08231838 는 버스정류장 반경 500m 내의 카카오가 제공하는 시설들의 갯수 입니다.
bus_byApi = pd.read_pickle('./Bus_around_500_08231838.pickle')
bus_byApi.head(3)

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,BK9,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,0,0,0,8,0,1,0
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,0,0,0,0,3,0,0,0
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
# 모든 카테고리의 수를 종합한 칼럼 category_sum 칼럼은 추후 score 칼럼을 계산할 때 사용됩니다.
bus_byApi['category_sum'] = bus_byApi.loc[:,'MT1':].sum(axis=1)
bus_byApi.head(2)

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9,category_sum
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,0,0,8,0,1,0,17
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,0,0,0,3,0,0,0,12


# 3. 버스 정류장 - 공원 사이를 알아보기

버스 정류장과 공원 사이의 거리를 구하여, 구간 별로 나눈 범위 중 몇개가 있는지 구한다

> ## 3.1 버스 정류장과 공원 사이의 거리 구하기

In [12]:
# 찬안시의 도시 공원정보 데이터를 불러와
park = pd.read_csv('./충청남도_천안시_도시공원정보_20230711.csv',encoding='cp949')
park.head(2)

# 필요한 컬럼만 선택후
park_cp = park[['관리번호','공원명','위도','경도']]

# 좌표 계산을 위해, 위도와 경도 데이터를 하나로 묶어 location칼럼을 만듭니다.
park_cp['location'] = pd.Series(zip(park_cp['위도'],park_cp['경도']))
park_cp.head(1)



,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자
0,44131-00043,용곡공원(용곡4),어린이공원,NaN,충청남도 천안시 동남구 용곡동 597,36.787417,127.135552,2300.0,NaN,조합놀이대,NaN,NaN,NaN,1993-02-18,충청남도 천안시청,041-521-2772,2023-07-11
1,44131-00044,눈들공원(용곡1),어린이공원,NaN,충청남도 천안시 동남구 용곡동 601,36.792611,127.139216,1500.0,NaN,조합놀이대+흔들놀이,NaN,NaN,NaN,1987-06-12,충청남도 천안시청,041-521-2772,2023-07-11


,관리번호,공원명,위도,경도,location
0,44131-00043,용곡공원(용곡4),36.787417,127.135552,"(36.78741709, 127.1355518)"


In [87]:
# 천안시의 버스 정류장과의 공원과의 거리 계산을 위해, 버스정류장 데이터를 불러와

bus = pd.read_pickle('./Bus_around_500_08231838.pickle')

# 필요한 칼럼만 추출하고,
bus_cp = bus[['정류장번호','정류장명','위도','경도']]
bus_cp.reset_index(drop=True, inplace=True)

# 좌표 계산을 위해 location 칼럼을 만듭니다.
bus_cp['location'] = pd.Series(zip(bus_cp['위도'],bus_cp['경도']))
bus_cp.head()

,정류장번호,정류장명,위도,경도,location
0,ASB285000077,성환8리,36.921468,127.122774,"(36.92146807, 127.12277401)"
1,ASB285000078,송덕,36.923783,127.118976,"(36.92378302, 127.11897568)"
2,ASB285000079,국립축산과학원,36.925931,127.111136,"(36.92593139, 127.11113568)"
3,ASB285000094,성환8리,36.921479,127.122505,"(36.9214794, 127.1225049)"
4,ASB285000095,성환터미널,36.917221,127.132732,"(36.91722136, 127.13273235)"


In [88]:
# 출력 예시를 위해 2개의 데이터만 선택함!
park_cp2 = park_cp.copy().head(2)
bus_cp2 = bus_cp.copy().head(2)

# x : 경도, y : 위도
def calculate_distance(bus_cp, company_house):
    import geopy.distance

    data_list = []

    for idx in tqdm(range(len(bus_cp))):
        for jdx in range(len(company_house)):
            
            distance = geopy.distance.distance(bus_cp.at[idx,'location'],company_house.at[jdx,'location']).km

            data = {'정류장번호':bus_cp.at[idx,'정류장번호'],'정류장명':bus_cp.at[idx,'정류장명'],'location':bus_cp.at[idx,'location'],
            '공원명':company_house.at[jdx,'공원명'],'공원좌표':company_house.at[jdx,'location'],'공원관리번호':company_house.at[jdx,'관리번호'],'distance':distance}
            
            data_list.append(data)

    data_df = pd.DataFrame(data_list)
    return data_df
    
# company_house.reset_index(drop=True,inplace=True)
bus_Around_company_house = calculate_distance(bus_cp2,park_cp2)
bus_Around_company_house

100%|██████████| 2/2 [00:00<00:00, 1999.19it/s]


,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",용곡공원(용곡4),"(36.78741709, 127.1355518)",44131-00043,14.919877
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",눈들공원(용곡1),"(36.79261113, 127.1392161)",44131-00044,14.374872
2,ASB285000078,송덕,"(36.92378302, 127.11897568)",용곡공원(용곡4),"(36.78741709, 127.1355518)",44131-00043,15.205229
3,ASB285000078,송덕,"(36.92378302, 127.11897568)",눈들공원(용곡1),"(36.79261113, 127.1392161)",44131-00044,14.668276


In [89]:
bus_park_distance = bus_Around_company_house.copy()
bus_park_distance

,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",용곡공원(용곡4),"(36.78741709, 127.1355518)",44131-00043,14.919877
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",눈들공원(용곡1),"(36.79261113, 127.1392161)",44131-00044,14.374872
2,ASB285000078,송덕,"(36.92378302, 127.11897568)",용곡공원(용곡4),"(36.78741709, 127.1355518)",44131-00043,15.205229
3,ASB285000078,송덕,"(36.92378302, 127.11897568)",눈들공원(용곡1),"(36.79261113, 127.1392161)",44131-00044,14.668276


In [ ]:
# 이후 거리 계산한 파일을 저장
# bus_park_distance.to_pickle('./버스정류장과공원사이의거리.pickle')


> ## 3.2 거리간 구간 나누기

In [90]:
bus_park = pd.read_pickle('./버스정류장과공원사이의거리.pickle')
bus_park

,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",용곡공원(용곡4),"(36.78741709, 127.1355518)",44131-00043,14.919877
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",눈들공원(용곡1),"(36.79261113, 127.1392161)",44131-00044,14.374872
2,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성소공원,"(36.81341463, 127.159059)",44131-00045,12.420054
3,ASB285000077,성환8리,"(36.92146807, 127.12277401)",원성공원(원성2),"(36.80788022, 127.1636754)",44131-00046,13.122437
4,ASB285000077,성환8리,"(36.92146807, 127.12277401)",꽃재공원(원성4),"(36.80085077, 127.1616121)",44131-00047,13.826295
...,...,...,...,...,...,...,...
765343,GGB500000184,둔포,"(36.9302833, 127.0387333)",백석공원,"(36.83303341, 127.1090661)",44133-00098,12.481557
765344,GGB500000184,둔포,"(36.9302833, 127.0387333)",두정10공원,"(36.82681344, 127.1507137)",44133-00099,15.215725
765345,GGB500000184,둔포,"(36.9302833, 127.0387333)",성성4지구 어린이공원,"(36.84053144, 127.1368524)",44133-00100,13.255707
765346,GGB500000184,둔포,"(36.9302833, 127.0387333)",문성소공원,"(36.80968397, 127.1542359)",44131-00065,16.887250


* class_A : 버스정류장과 타겟과의 거리가 0이상 0.5 이하에 속하는 것

* class_B : 버스정류장과 타겟과의 거리가 0.5초과 1 이하에 속하는 것

        (거리 단위는 km)

In [91]:
# 버스 정류장과 공원 사이의 거리가 0<=distance<=0.5, 0.5<distance<=1 의 갯수를 각각 구한다.
bus_park_class_A = bus_park[(0<=bus_park['distance'])&(bus_park['distance']<=0.5)]
bus_park_class_B = bus_park[(0.5<bus_park['distance'])&(bus_park['distance']<=1)]
bus_park_class_A.head(2)
bus_park_class_B.head(2)

,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
1493,ASB285000099,삼풍아파트,"(36.9125166, 127.1309215)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.317816
2933,ASB285000165,쌍용동일아파트,"(36.7848, 127.11328)",쌍용10공원,"(36.78655184, 127.1168556)",44133-00077,0.373709


,정류장번호,정류장명,location,공원명,공원좌표,공원관리번호,distance
785,ASB285000095,성환터미널,"(36.91722136, 127.13273235)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.568513
962,ASB285000096,성신초등학교,"(36.9176694, 127.1277947)",성환4공원,"(36.91229381, 127.1344771)",44133-00053,0.842896


In [92]:
# 공원거리구분_A : 0.5<distance<=1 
class_A_df= pd.DataFrame(bus_park_class_A['정류장번호'].value_counts()).reset_index()
class_A_df.columns =['정류장번호','공원거리구분_A']
class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
class_A_df.reset_index(drop=True,inplace=True)
class_A_df

# 공원거리구분_B : 0.5<distance<=1 
class_B_df= pd.DataFrame(bus_park_class_B['정류장번호'].value_counts()).reset_index()
class_B_df.columns =['정류장번호','공원거리구분_B']
class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
class_B_df.reset_index(drop=True,inplace=True)
class_B_df

,정류장번호,공원거리구분_A
0,ASB285000099,1
1,ASB285000165,1
2,ASB285000287,1
3,ASB285000649,2
4,ASB285000650,2
...,...,...
781,GGB277102211,1
782,GGB285000034,1
783,GGB285500010,4
784,GGB285500020,6


,정류장번호,공원거리구분_B
0,ASB285000095,1
1,ASB285000096,1
2,ASB285000097,1
3,ASB285000098,1
4,ASB285000100,1
...,...,...
1008,GGB285500010,4
1009,GGB285500020,8
1010,GGB285500030,8
1011,GGB288500130,8


In [93]:
bus_station= bus_byApi.copy()
bus_station_merge_class_A = pd.merge(bus_station,class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print('테이블간 결합됨 :',len(bus_station)==len(bus_station_merge_class_A))

bus_station_merge_class_AB = pd.merge(bus_station_merge_class_A,class_B_df, on='정류장번호',how='left')


print('테이블간 결합됨 :',len(bus_station)==len(bus_station_merge_class_AB))

테이블간 결합됨 : True
테이블간 결합됨 : True


In [94]:
bus_station_merge_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_class_AB['공원거리구분_A'] = bus_station_merge_class_AB['공원거리구분_A'].fillna(0)
bus_station_merge_class_AB['공원거리구분_B'] = bus_station_merge_class_AB['공원거리구분_B'].fillna(0)

bus_station_merge_class_AB['공원거리구분_A'] = bus_station_merge_class_AB['공원거리구분_A'].astype('int')
bus_station_merge_class_AB['공원거리구분_B'] = bus_station_merge_class_AB['공원거리구분_B'].astype('int')

bus_station_merge_class_AB.head(1)


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,PO3,AT4,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,8,0,1,0,17,NaN,NaN


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,PO3,AT4,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,8,0,1,0,17,0,0


# 4. 버스 정류장과 거주지, 회사의 갯수 구하기

위의 작업과 마찬가지로, 버스정류장과 거주지,회사간의 거리를 구하여 그 거리를 구간별로 나눠 갯수를 구하는 작업을 진행합니다.


> ## 4.1 버스 정류장과 거주지, 회사의 거리 구하기

> ### 4.1.1 회사 데이터 거리 구하기

In [86]:
cmp1 = pd.read_pickle('./제조업체데이터0819.pickle')
cmp2 = pd.read_pickle('./산업단지byAPI_0817.pickle')
cmp2.rename(columns={'회사명':'회사'},inplace=True)
cmp1['회사구분'] ='제조업체'
cmp2['회사구분'] ='산업단지'
cmp1 = cmp1[['회사','byAPI','회사구분']]
cmp2 = cmp2[['회사','byAPI','회사구분']]
cmpAll = pd.concat([cmp1,cmp2],axis=0)
cmp1.head()
cmp2.head()
cmpAll.head()

,회사,byAPI,회사구분
0,(주)동아수지산업,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체
1,KY컴퍼니,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체
2,N&P시스템,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체
3,RP이엔지,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체
4,남주테크,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체


In [87]:
cmpAll.reset_index(inplace=True,drop=True)


cmpAll['x'] = cmpAll['byAPI'].apply(lambda elm: elm.get('x', {}))
cmpAll['y'] = cmpAll['byAPI'].apply(lambda elm: elm.get('y', {}))
cmpAll.head(2)

cmpAll['x'] = cmpAll['x'].astype('float64')
cmpAll['y'] = cmpAll['y'].astype('float64')
cmpAll['location'] = pd.Series(zip(cmpAll['y'],cmpAll['x']))
cmpAll.head(2)

,회사,byAPI,회사구분,x,y,location
0,(주)동아수지산업,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체,127.103428,36.865651,"(36.8656506084092, 127.103428417573)"
1,KY컴퍼니,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체,127.104524,36.866293,"(36.8662931269303, 127.104524303521)"


> ### 4.1.2 거주지 데이터 거리 구하기

In [88]:
# 거주기 데이터 불러오기
house = pd.read_pickle('./addressByAPIpickle08191010.pickle')
house.head(2)

# 계산에 필요한 컬럼만 추출합니다.
house_cp = house[['건물명','byAPI']]
house_cp.head(2)


,건물명,byAPI
0,천안맨션아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...
1,미도아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...


In [89]:
house_cp['x'] = house_cp['byAPI'].apply(lambda elm : elm.get('x'))
house_cp['y'] = house_cp['byAPI'].apply(lambda elm : elm.get('y'))
house_cp['x'] = house_cp['x'].astype('float64')
house_cp['y'] = house_cp['y'].astype('float64')

def create_location(row):
    return (row['y'], row['x'])

house_cp['location'] = house_cp.apply(create_location, axis=1)



In [90]:
cmpAll.rename(columns = {'회사':'이름','회사구분':'종류'},inplace=True)
house_cp.rename(columns = {'건물명':'이름'},inplace=True)

house_cp['종류'] ='집'
cmpAll.head(2)
house_cp.head(2)

,이름,byAPI,x,y,location,종류
0,천안맨션아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 4...,127.148220,36.811988,"(36.811988462486, 127.148220373057)",집
1,미도아파트,{'address': {'address_name': '충남 천안시 동남구 대흥동 1...,127.147978,36.806906,"(36.8069064103081, 127.147978496779)",집


In [91]:
company_house = pd.concat([cmpAll,house_cp],axis=0)
company_house.head()

,이름,byAPI,종류,x,y,location
0,(주)동아수지산업,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체,127.103428,36.865651,"(36.8656506084092, 127.103428417573)"
1,KY컴퍼니,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체,127.104524,36.866293,"(36.8662931269303, 127.104524303521)"
2,N&P시스템,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체,127.103433,36.865970,"(36.8659698650003, 127.103432997372)"
3,RP이엔지,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체,127.102342,36.865351,"(36.8653507611359, 127.102341642294)"
4,남주테크,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,제조업체,127.102750,36.864648,"(36.8646478195452, 127.102749590215)"


In [92]:
# 거리 계산을 위해, 버스 정류장 데이터 불러오기
bus = pd.read_pickle('./Bus_around_500_08231838.pickle')
bus_cp = bus[['정류장번호','정류장명','위도','경도']]
bus_cp.reset_index(drop=True, inplace=True)
bus_cp['location'] = pd.Series(zip(bus_cp['위도'],bus_cp['경도']))
bus_cp.head()

,정류장번호,정류장명,위도,경도,location
0,ASB285000077,성환8리,36.921468,127.122774,"(36.92146807, 127.12277401)"
1,ASB285000078,송덕,36.923783,127.118976,"(36.92378302, 127.11897568)"
2,ASB285000079,국립축산과학원,36.925931,127.111136,"(36.92593139, 127.11113568)"
3,ASB285000094,성환8리,36.921479,127.122505,"(36.9214794, 127.1225049)"
4,ASB285000095,성환터미널,36.917221,127.132732,"(36.91722136, 127.13273235)"


In [106]:
bus_cp = bus_cp.head(2)
company_house = company_house.head(2)
def calculate_distance(bus_cp, company_house):
    data_list = []
    import geopy.distance
    for idx in tqdm(range(len(bus_cp))):
        for jdx in range(len(company_house)):
            
            distance = geopy.distance.distance(bus_cp.at[idx,'location'],company_house.at[jdx,'location']).km

            data = {'정류장번호':bus_cp.at[idx,'정류장번호'],'정류장명':bus_cp.at[idx,'정류장명'],
           
            'location':bus_cp.at[idx,'location'],'이름':company_house.at[jdx,'이름'],'byAPI':company_house.at[jdx,'byAPI'],
            'house_loc':company_house.at[jdx,'location'],'종류':company_house.at[jdx,'종류'],'distance':distance}
            
            data_list.append(data)

    data_df = pd.DataFrame(data_list)
    return data_df
    
company_house.reset_index(drop=True,inplace=True)
bus_Around_company_house = calculate_distance(bus_cp,company_house)
bus_Around_company_house

100%|██████████| 2/2 [00:00<00:00, 999.95it/s]


,정류장번호,정류장명,location,이름,byAPI,house_loc,종류,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",(주)동아수지산업,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8656506084092, 127.103428417573)",제조업체,6.429916
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",KY컴퍼니,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8662931269303, 127.104524303521)",제조업체,6.335470
2,ASB285000078,송덕,"(36.92378302, 127.11897568)",(주)동아수지산업,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8656506084092, 127.103428417573)",제조업체,6.598445
3,ASB285000078,송덕,"(36.92378302, 127.11897568)",KY컴퍼니,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8662931269303, 127.104524303521)",제조업체,6.508714


In [ ]:
# 버스 정류장과 회사,거주지 간의 거리를 구한 데이터 셋인 bus_Around_company_house 를 저장합니다.
# bus_Around_company_house.to_pickle('./bus_Around_company_house.pickle')

> ## 4.2 거리간 구간 나누기

In [34]:
bus_CH = pd.read_pickle('./bus_Around_company_house.pickle')
bus_CH.head(2)

,정류장번호,정류장명,coord2address,주변시설의총합,location,이름,byAPI,house_loc,종류,distance
0,ASB285000077,성환8리,"{'road_address': None, 'address': {'address_na...",550,"(36.92146807, 127.12277401)",(주)동아수지산업,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8656506084092, 127.103428417573)",제조업체,6.429916
1,ASB285000077,성환8리,"{'road_address': None, 'address': {'address_na...",550,"(36.92146807, 127.12277401)",KY컴퍼니,{'address': {'address_name': '충남 천안시 서북구 직산읍 마...,"(36.8662931269303, 127.104524303521)",제조업체,6.335470


In [109]:
# A, B 로 구간 나누기
bus_CH_A = bus_CH[(0<=bus_CH['distance'])&(bus_CH['distance']<=0.5)]
bus_CH_B = bus_CH[(0.5<bus_CH['distance'])&(bus_CH['distance']<=1)]



In [110]:
# # class_A : 버스 정류장과 집,회사 사이의 거리가 0<=distance<=0.5
bus_CH_class_A_df= pd.DataFrame(bus_CH_A['정류장번호'].value_counts()).reset_index()
bus_CH_class_A_df.columns =['정류장번호','CH거리구분_A']
bus_CH_class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_CH_class_A_df.reset_index(drop=True,inplace=True)
bus_CH_class_A_df.head(2)

# # class_B : 버스 정류장과 집,회사 사이의 거리가 0.5< distance<=1
bus_CH_class_B_df= pd.DataFrame(bus_CH_B['정류장번호'].value_counts()).reset_index()
bus_CH_class_B_df.columns =['정류장번호','CH거리구분_B']
bus_CH_class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_CH_class_B_df.reset_index(drop=True,inplace=True)
bus_CH_class_B_df.head(2)

,정류장번호,CH거리구분_A
0,ASB285000077,5
1,ASB285000078,11


,정류장번호,CH거리구분_B
0,ASB285000077,25
1,ASB285000078,15


In [111]:
# 테이블을 병합하기위해, copy본과 결합
bus_station_merge_class_AB_cp = bus_station_merge_class_AB.copy()

bus_station_merge_CH_class_A = pd.merge(bus_station_merge_class_AB_cp,bus_CH_class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_CH_class_A))

bus_station_merge_CH_class_AB = pd.merge(bus_station_merge_CH_class_A,bus_CH_class_B_df, on='정류장번호',how='left')

print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_CH_class_AB))

bus_station_merge_CH_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_CH_class_AB['CH거리구분_A'] = bus_station_merge_CH_class_AB['CH거리구분_A'].fillna(0)
bus_station_merge_CH_class_AB['CH거리구분_B'] = bus_station_merge_CH_class_AB['CH거리구분_B'].fillna(0)

bus_station_merge_CH_class_AB['CH거리구분_A'] = bus_station_merge_CH_class_AB['CH거리구분_A'].astype('int')
bus_station_merge_CH_class_AB['CH거리구분_B'] = bus_station_merge_CH_class_AB['CH거리구분_B'].astype('int')

# 병합된 데이터 셋
bus_station_merge_CH_class_AB.head(1)


테이블간 결합됨 True
테이블간 결합됨 True


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,8,0,1,0,17,0,0,5.0,25.0


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,AD5,FD6,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,8,0,1,0,17,0,0,5,25


# 5. 버스 정류장과 하천과의 거리 갯수 구하기
위의 작업들과 동일하며, 버스정류장과 하천과의 거리를 구하여 구간별로 나눠 갯수를 센다

> ## 5.1 버스 정류장과 하천간의 거리 구하기

In [112]:
bus = pd.read_pickle('./Bus_around_500_08231838.pickle')
bus_cp = bus[['정류장번호','정류장명','위도','경도']]
bus_cp.reset_index(drop=True, inplace=True)

bus_cp['location'] = pd.Series(zip(bus_cp['위도'],bus_cp['경도']))
bus_cp.head()

,정류장번호,정류장명,위도,경도,location
0,ASB285000077,성환8리,36.921468,127.122774,"(36.92146807, 127.12277401)"
1,ASB285000078,송덕,36.923783,127.118976,"(36.92378302, 127.11897568)"
2,ASB285000079,국립축산과학원,36.925931,127.111136,"(36.92593139, 127.11113568)"
3,ASB285000094,성환8리,36.921479,127.122505,"(36.9214794, 127.1225049)"
4,ASB285000095,성환터미널,36.917221,127.132732,"(36.91722136, 127.13273235)"


In [113]:
water = pd.read_csv('./전국하천위도경도_xy.csv')
water['location'] = pd.Series(zip(water['y'],water['x']))
water.head(2)

,A0,A1,A2,A3,x,y,location
0,710,64800004888020200241UJB4000001030,하천/용도구역,2023/03/16,127.908859,35.848407,"(35.8484069199, 127.9088593132)"
1,711,64800004888020200241UJB4000001031,하천/용도구역,2023/03/16,127.908469,35.841851,"(35.841851007, 127.9084686265)"


In [114]:
def calculate_distance(bus_cp, company_house):
    import geopy.distance

    data_list = []

    for idx in tqdm(range(len(bus_cp))):
        for jdx in range(len(company_house)):
            
            distance = geopy.distance.distance(bus_cp.at[idx,'location'],company_house.at[jdx,'location']).km

            data = {'정류장번호':bus_cp.at[idx,'정류장번호'],'정류장명':bus_cp.at[idx,'정류장명'],'location':bus_cp.at[idx,'location'],
            'A1':company_house.at[jdx,'A1'],'하천좌표':company_house.at[jdx,'location'],'distance':distance}
            
            data_list.append(data)

    data_df = pd.DataFrame(data_list)
    return data_df
    

# 예시를 위해 2개의 데이터만 넣습니다.
bus_cp = bus_cp.head(2)
water_cp = water.head(2)
bus_target_distance = calculate_distance(bus_cp,water_cp)
bus_target_distance

100%|██████████| 2/2 [00:00<00:00, 1999.67it/s]


,정류장번호,정류장명,location,A1,하천좌표,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001030,"(35.8484069199, 127.9088593132)",138.392944
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001031,"(35.841851007, 127.9084686265)",139.003014
2,ASB285000078,송덕,"(36.92378302, 127.11897568)",64800004888020200241UJB4000001030,"(35.8484069199, 127.9088593132)",138.787206
3,ASB285000078,송덕,"(36.92378302, 127.11897568)",64800004888020200241UJB4000001031,"(35.841851007, 127.9084686265)",139.396821


In [ ]:
# 버스 정류장과 하천과의 거리를 구한 데이터 셋인 bus_target_distance 을 저장합니다.
# bus_target_distance.to_pickle('./버스정류장과하천과의거리.pickle')


> ## 5.2 거리간 구간 나누기

In [116]:
bus_stream = pd.read_pickle('./버스정류장과하천과의거리.pickle')
bus_stream.head(2)

,정류장번호,정류장명,location,A1,하천좌표,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001030,"(35.8484069199, 127.9088593132)",138.392944
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001031,"(35.841851007, 127.9084686265)",139.003014


In [117]:
bus_stream_A  = bus_stream[(0<=bus_stream['distance'])&(bus_stream['distance']<=0.5)]
bus_stream_B  = bus_stream[(0.5<bus_stream['distance'])&(bus_stream['distance']<=1)]
bus_stream_A.head(2)
bus_stream_B.head(2)


# # class_A : 버스 정류장과 집,회사 사이의 거리가 0<=distance<=0.5
bus_stream_class_A_df= pd.DataFrame(bus_stream_A['정류장번호'].value_counts()).reset_index()
bus_stream_class_A_df.columns =['정류장번호','하천거리구분_A']
bus_stream_class_A_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_stream_class_A_df.reset_index(drop=True,inplace=True)
bus_stream_class_A_df.head(2)

# # class_B : 버스 정류장과 집,회사 사이의 거리가 0.5<distance<=1
bus_stream_class_B_df= pd.DataFrame(bus_stream_B['정류장번호'].value_counts()).reset_index()
bus_stream_class_B_df.columns =['정류장번호','하천거리구분_B']
bus_stream_class_B_df.sort_values('정류장번호',ascending=True,inplace=True)
bus_stream_class_B_df.reset_index(drop=True,inplace=True)
bus_stream_class_B_df.head(2)

,정류장번호,정류장명,location,A1,하천좌표,distance
7736,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.398270
37124,ASB285000094,성환8리,"(36.9214794, 127.1225049)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.421355


,정류장번호,정류장명,location,A1,하천좌표,distance
17532,ASB285000078,송덕,"(36.92378302, 127.11897568)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.813301
46920,ASB285000095,성환터미널,"(36.91722136, 127.13273235)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.613733


,정류장번호,하천거리구분_A
0,ASB285000077,1
1,ASB285000094,1


,정류장번호,하천거리구분_B
0,ASB285000078,1
1,ASB285000095,1


In [118]:
# 테이블을 병합하기위해, copy본과 결합
bus_station_merge_class_AB_cp = bus_station_merge_CH_class_AB.copy()

bus_station_merge_stream_class_A = pd.merge(bus_station_merge_class_AB_cp,bus_stream_class_A_df,on='정류장번호',how='left')

# 천안시의 버스정류장 데이터와, 천안시 버스정류장-공원거리구분 A 데이터가 잘 병합됬었는지 데이터 갯수를 통해 확인
print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_stream_class_A))

bus_station_merge_stream_class_AB = pd.merge(bus_station_merge_stream_class_A,bus_stream_class_B_df, on='정류장번호',how='left')

print('테이블간 결합됨',len(bus_station)==len(bus_station_merge_stream_class_AB))

bus_station_merge_stream_class_AB.head(1)
# 테이블 병합이후, NaN값은 0으로 채우고, 병합한 컬럼의 자료형은 int 로 변경
bus_station_merge_stream_class_AB['하천거리구분_A'] = bus_station_merge_stream_class_AB['하천거리구분_A'].fillna(0)
bus_station_merge_stream_class_AB['하천거리구분_B'] = bus_station_merge_stream_class_AB['하천거리구분_B'].fillna(0)

bus_station_merge_stream_class_AB['하천거리구분_A'] = bus_station_merge_stream_class_AB['하천거리구분_A'].astype('int')
bus_station_merge_stream_class_AB['하천거리구분_B'] = bus_station_merge_stream_class_AB['하천거리구분_B'].astype('int')

bus_station_merge_stream_class_AB.head(1)


테이블간 결합됨 True
테이블간 결합됨 True


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,1,0,17,0,0,5,25,1.0,NaN


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CE7,HP8,PM9,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,1,0,17,0,0,5,25,1,0


In [ ]:

bus_station_merge_stream_class_AB.head()
# 최종적으로 사용하게 될 데이터 셋인 bus_station_merge_stream_class_AB 을 저장합니다.
# bus_station_merge_stream_class_AB.to_pickle('./최종사용데이터.pickle')


> ## 5.3 특정 8개 지역 고려하기

태블로 시각화를 통해 얻은 결과를 바탕으로, 천안시의 주요 인구 유동 지역인 '백석동','성정동','신안동','부성동','불당동','쌍용동','청룡동','신방동' 으로 총 8지역의 후보를 정하였다.

In [18]:
final_data = pd.read_pickle('./최종사용데이터_25개구.pickle')

selected_data=final_data[final_data.대분류.isin(['백석동','성정동','신안동','부성동','불당동','쌍용동','청룡동','신방동'])]
selected_data.head()

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,category_sum,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,score,읍/면/동,대분류
16,ASB285000165,쌍용동일아파트,36.78480,127.11328,1,4,10,2,19,1,...,85,1,3,2,4,0,0,98,신방동,신방동
17,ASB285000287,백석산업단지입구,36.82298,127.10901,1,16,7,0,29,10,...,177,1,5,306,204,0,0,1000,불당동,불당동
18,ASB285000649,방죽안오거리,36.81744,127.15115,1,18,3,5,61,15,...,394,2,15,10,80,0,0,513,신부동,신안동
19,ASB285000650,방죽안오거리,36.81758,127.15149,1,18,3,5,62,15,...,430,2,16,10,83,0,0,553,신부동,신안동
34,ASB285000686,종합터미널,36.81901,127.15656,3,33,13,2,108,25,...,886,5,7,17,58,0,0,995,신부동,신안동


# 6. Score 계산하기

반경 500m이내의 시설의 갯수를 종합한 수와, 공원거리,회사와거주지의거리, 하천거리의 정보 값들의 범위를 맞추기 위하여

스케일링작업을 진행합니다.

> ## 6.1 스케일링

In [19]:
# ToDo. "최종사용데이터" 파일을 통해 스케일링항 코드 추가하기


data = selected_data.copy()

# 8개의 지역만 있는 데이터를 스케일링함


# 추가 검토 필요
##category_sum 스케일링
data['minmax_category']=(data.category_sum-min(data.category_sum))/max(data.category_sum)-min(data.category_sum)

##공원거리 스케일링
data['minmax_공원']=((data.공원거리구분_A+data.공원거리구분_B)-min(data.공원거리구분_A+data.공원거리구분_B))/max(data.공원거리구분_A+data.공원거리구분_B)-min(data.공원거리구분_A+data.공원거리구분_B)

##하천거리 스케일링
data['minmax_하천']=((data.하천거리구분_A+data.하천거리구분_B)-min(data.하천거리구분_A+data.하천거리구분_B))/max(data.하천거리구분_A+data.하천거리구분_B)-min(data.하천거리구분_A+data.하천거리구분_B)

##주거회사거리 스케일링
data['minmax_주거회사']=((data.CH거리구분_A+data.CH거리구분_B)-min(data.CH거리구분_A+data.CH거리구분_B))/max(data.CH거리구분_A+data.CH거리구분_B)-min(data.CH거리구분_A+data.CH거리구분_B)

##minmax 스케일링 후 score 합 
data['minmax_sum_score']=data.minmax_category+data.minmax_공원+data.minmax_하천+data.minmax_주거회사





data.head(2)

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,하천거리구분_A,하천거리구분_B,score,읍/면/동,대분류,minmax_category,minmax_공원,minmax_하천,minmax_주거회사,minmax_sum_score
16,ASB285000165,쌍용동일아파트,36.78480,127.11328,1,4,10,2,19,1,...,0,0,98,신방동,신방동,0.070774,0.181818,0.0,0.010601,0.263193
17,ASB285000287,백석산업단지입구,36.82298,127.10901,1,16,7,0,29,10,...,0,0,1000,불당동,불당동,0.147377,0.272727,0.0,0.901060,1.321165


> ## 6.2 스코어 계산하기

In [20]:
##minmax 스케일링 후 가중치 부여 score 합 
data['score7']=(data.minmax_category)+data.minmax_공원+data.minmax_하천+(data.minmax_주거회사)*7
data.head(2)

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,하천거리구분_B,score,읍/면/동,대분류,minmax_category,minmax_공원,minmax_하천,minmax_주거회사,minmax_sum_score,score7
16,ASB285000165,쌍용동일아파트,36.78480,127.11328,1,4,10,2,19,1,...,0,98,신방동,신방동,0.070774,0.181818,0.0,0.010601,0.263193,0.326797
17,ASB285000287,백석산업단지입구,36.82298,127.10901,1,16,7,0,29,10,...,0,1000,불당동,불당동,0.147377,0.272727,0.0,0.901060,1.321165,6.727525


In [13]:
# data.to_pickle('./8개구_스케일링데이터_0829.pickle')

In [14]:

# 스케일랑과 그 점수를 합산한 데이터 셋을 저장합니다.
# 데스크탑에서.
# data.to_pickle(r'E:\제출용데이터dir\천안시데이터\preprocessing\최종사용데이터_8개구_스케일링.pickle')

# 7. 클러스터링 진행하기

In [21]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [22]:
# scaled_data = pd.read_pickle(r'./8개구_스케일링데이터_0829.pickle')



scaled_data = data.copy()


print(len(scaled_data))


# 정류장명 중복인것은, minmax_sum_score 값이 큰것만 남기고 지우기
scaled_data_cp = scaled_data.copy()

target_col = 'score7'

scaled_data_cp.sort_values(by=target_col,inplace=True,ascending=False)
scaled_data_cp.drop_duplicates(subset='정류장명',keep='first',inplace=True)



# 다시 정렬 
scaled_data_cp.sort_values(by=target_col,inplace=True,ascending=False)
print(len(scaled_data_cp))



720
404


In [23]:
# score를 기준으로 필터링하기.
filtered_score = scaled_data_cp[target_col].median()
selected_rows = scaled_data_cp[scaled_data_cp[target_col] >= filtered_score]
print(filtered_score)
print(len(selected_rows))



1.3610643555683466
202


In [24]:
# 클러스터링을 하기위해, 데이터 카피
data = selected_rows.copy()

In [25]:
from sklearn.cluster import KMeans


coordinates = data[['위도', '경도']]
num_clusters = 30 # You can change this number
kmeans = KMeans(n_clusters=num_clusters,random_state=98)

kmeans.fit(coordinates)

data['cluster'] = kmeans.labels_


print("Cluster Centers:")
print(kmeans.cluster_centers_)
print(data['cluster'].value_counts())




Cluster Centers:
[[ 36.83425733 127.13493704]
 [ 36.80552624 127.13033647]
 [ 36.79839827 127.15998508]
 [ 36.82396711 127.11168476]
 [ 36.82156985 127.14226079]
 [ 36.85513433 127.124047  ]
 [ 36.80669141 127.11102789]
 [ 36.80465739 127.11960814]
 [ 36.79519725 127.13094855]
 [ 36.81823571 127.15477733]
 [ 36.81209613 127.14103304]
 [ 36.83248521 127.11458862]
 [ 36.83293713 127.14332673]
 [ 36.81948431 127.11925641]
 [ 36.82563531 127.1367811 ]
 [ 36.79068249 127.15554785]
 [ 36.81509097 127.10891843]
 [ 36.79938382 127.11738793]
 [ 36.7933637  127.12360741]
 [ 36.79263834 127.14248635]
 [ 36.81696169 127.13407929]
 [ 36.8300711  127.1071625 ]
 [ 36.82395204 127.12165733]
 [ 36.82425914 127.15428331]
 [ 36.82201295 127.10528406]
 [ 36.7878218  127.1290401 ]
 [ 36.83106298 127.12407219]
 [ 36.79200106 127.16171843]
 [ 36.79998184 127.12573171]
 [ 36.85055001 127.11794465]]
4     14
8     10
14    10
7      9
6      8
0      8
12     8
18     8
9      8
10     8
11     8
2      8
24  

In [26]:
data

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,score,읍/면/동,대분류,minmax_category,minmax_공원,minmax_하천,minmax_주거회사,minmax_sum_score,score7,cluster
903,CAB285000993,브라운스톤아파트,36.830031,127.114282,0,4,12,0,5,2,...,820,백석동,백석동,0.059950,0.454545,0.0,0.994700,1.509195,7.477393,11
1976,CAB285071465,백석이수아파트입구,36.830135,127.116302,0,3,14,0,12,4,...,704,백석동,백석동,0.027477,0.500000,0.0,0.989399,1.516876,7.453272,11
1773,CAB285002238,한국더첼,36.830729,127.113985,0,2,10,0,5,2,...,760,백석동,백석동,0.015820,0.409091,0.0,0.996466,1.421377,7.400176,11
904,CAB285000995,벽산블루밍아파트,36.826201,127.113307,0,8,15,1,10,2,...,1117,백석동,백석동,0.109076,0.500000,0.0,0.961131,1.570207,7.336991,3
1774,CAB285002239,제네시스,36.831043,127.112442,0,2,6,0,4,1,...,798,백석동,백석동,0.010824,0.318182,0.0,1.000000,1.329006,7.329006,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,CAB285000873,나사렛대학교,36.795430,127.117816,0,19,22,2,47,20,...,465,쌍용동,쌍용동,0.336386,0.636364,0.0,0.058304,1.031054,1.380877,17
2060,CAB285071558,현대6차아파트,36.791416,127.124305,1,19,23,3,75,15,...,227,쌍용동,쌍용동,0.127394,0.681818,0.0,0.081272,0.890484,1.378117,18
1912,CAB285010124,"리슈빌,아이파크",36.834181,127.123176,2,10,14,0,30,6,...,145,백석동,백석동,0.051624,0.545455,0.0,0.109541,0.706619,1.363863,26
1535,CAB285001894,동아제약,36.840466,127.112500,0,2,0,0,0,6,...,133,차암동,부성동,0.013322,0.272727,0.0,0.153710,0.439760,1.362021,11


In [27]:
# 클러스터 중심 시각화를 위한 cluster_cneters 데이터 셋 만들기.
cluster_centers_df = pd.DataFrame(kmeans.cluster_centers_, columns=['위도', '경도'])
cluster_centers_df['cluster_label'] = range(num_clusters)
cluster_centers_df.head(2)

,위도,경도,cluster_label
0,36.834257,127.134937,0
1,36.805526,127.130336,1


In [28]:
# 클러스터안에서, 중복인 데이터 지우기
data.sort_values(by=target_col,ascending=False,inplace=True)
# data.head()
data.drop_duplicates(subset='cluster',keep='first',inplace=True)
print(len(data))


30


In [ ]:
data

# 8. 시각화

클러스터링한 결과를 통해, 따릉이를 거치할 버스 정류장의 후보군을 선정합니다.

>## potly를 통한 시각화

In [29]:
import plotly.graph_objects as go
import plotly.express as px

In [50]:
# bus_station_Main = pd.read_pickle('./Bus_Main.pickle')

In [30]:
# 시각화를 위해 데이터 카피하기
data_cp = data.copy()


In [31]:
# 마커 사이즈를 위한, 마커 크기 구하기

c_col ='quartile classification'

min_score = data_cp[target_col].min()
max_score = data_cp[target_col].max()
data_cp['for_marker_size'] = 1 + ((data_cp[target_col] - min_score) / (max_score - min_score)) * 9
data_cp['for_marker_size'] = data_cp['for_marker_size'].round()

data_cp['default_marker_size'] = 100

# df = data_bus_station_merge.copy()


In [27]:
# # 천안시의 자전거 정류소와 비교해볼 천안시 자전거 정류장 데이터 셋
# ground_truth = pd.read_csv('./천안시예상정류장설치장소CSV.csv',encoding='cp949')
# ground_truth.head()

,장소명,위도,경도,Unnamed: 3
0,천안 마치 에비뉴 파크,36.837876,127.119234,NaN
1,불당지구,36.810255,127.105926,NaN
2,천안시청,36.816130,127.113047,NaN
3,쌍용역,36.792908,127.120194,NaN
4,두정역,36.833788,127.148933,NaN


In [54]:
# data_cp['blue'] ='blue'

In [32]:
scaled_data['blue']='blue'
scaled_data['for_marker_size'] =1

In [67]:
len(data_cp)

30

In [33]:
bus_c_h = bus_CH.copy()
bus_c_h = bus_c_h.drop_duplicates(subset='이름')
bus_c_h['종류'].value_counts()
bus_c_h_visual = bus_c_h[['이름','house_loc','종류']]
bus_c_h_visual['위도'] = bus_c_h_visual['house_loc'].apply(lambda elm:elm[0])
bus_c_h_visual['경도'] = bus_c_h_visual['house_loc'].apply(lambda elm:elm[1])
bus_c_h_visual.head()

cmp_make = bus_c_h_visual[bus_c_h_visual['종류']=='제조업체']
cmp_facil = bus_c_h_visual[bus_c_h_visual['종류']=='생산업체']
house = bus_c_h_visual[bus_c_h_visual['종류']=='집']


NameError: name 'bus_CH' is not defined

In [124]:
house

,이름,house_loc,종류,위도,경도
3562,천안맨션아파트,"(36.811988462486, 127.148220373057)",집,36.811988,127.148220
3563,미도아파트,"(36.8069064103081, 127.147978496779)",집,36.806906,127.147978
3564,빌라,"(36.805337219138, 127.147903198814)",집,36.805337,127.147903
3565,반석빌라,"(36.8038565780048, 127.147051515516)",집,36.803857,127.147052
3566,성황비둘기아파트,"(36.8146026469712, 127.151812203465)",집,36.814603,127.151812
...,...,...,...,...,...
4273,희홍 밸러-뷰,"(36.8958265814396, 127.166867730409)",집,36.895827,127.166868
4276,e편한세상 스마일시티 2차,"(36.8474780033796, 127.107938789162)",집,36.847478,127.107939
4277,e편한세상 스마일시티,"(36.843910397879, 127.103628071614)",집,36.843910,127.103628
4278,한화꿈에그린스마일시티,"(36.8415664288631, 127.101659351039)",집,36.841566,127.101659


In [36]:

# 전체 버스 정류장의 위치 시각화
fig = px.scatter_mapbox(
    scaled_data,
    lat='위도',
    lon='경도',
    color='blue',  # Set marker color to blue
    size_max=5,      # Adjust point size
    zoom=9,           # Initial map zoom level
    hover_name='정류장명',
    size='for_marker_size'
)

# # Create larger red square markers for '위도' and '경도' from the 'park' DataFrame
# park_trace = go.Scattermapbox(
#     lat=park['위도'],
#     lon=park['경도'],
#     mode='markers',
#     marker=dict(size=5, color='red', symbol='circle'),  # Increase the size to 15
#     hovertext=park['공원명']
# )
# fig.add_trace(park_trace)

# ground_truth_trace = go.Scattermapbox(
#     lat=ground_truth['위도'],
#     lon=ground_truth['경도'],
#     mode='markers',
#     marker=dict(size=30, color='yellowgreen', symbol='circle'),
#     hovertext=ground_truth['장소명']  # Replace with the appropriate column from ground_truth
# )
# fig.add_trace(ground_truth_trace)


# 클러스터의 중심 좌표 시각화
cluster_centers_df_trace = go.Scattermapbox(
    lat = cluster_centers_df['위도'],
    lon = cluster_centers_df['경도'],
    mode='markers',
    marker = dict(size=15,color='black',symbol='circle'),
    hovertext=cluster_centers_df['cluster_label']
)
fig.add_trace(cluster_centers_df_trace)


# 따릉이 설치 장소로 고려할 장소 시각화
data_cp_trance = go.Scattermapbox(
    lat = data_cp['위도'],
    lon = data_cp['경도'],
    mode='markers',
    marker = dict(size=15,color='red',symbol='circle'),
    hovertext=data_cp['정류장명']
)
fig.add_trace(data_cp_trance)


# # 제조업체 회사 시각화
# cmp_make_trance = go.Scattermapbox(
#     lat = cmp_make['위도'],
#     lon = cmp_make['경도'],
#     mode='markers',
#     marker = dict(size=10,color='orange',symbol='circle'),
#     hovertext=cmp_make['이름']
# )
# fig.add_trace(cmp_make_trance)

# # 생산업체 회사 시각화
# cmp_facil_trance = go.Scattermapbox(
#     lat = cmp_facil['위도'],
#     lon = cmp_facil['경도'],
#     mode='markers',
#     marker = dict(size=10,color='purple',symbol='circle'),
#     hovertext=cmp_facil['이름']
# )
# fig.add_trace(cmp_facil_trance)

# # 거주지 위치 시각화
# house_trance = go.Scattermapbox(
#     lat = house['위도'],
#     lon = house['경도'],
#     mode='markers',
#     marker = dict(size=10,color='indigo',symbol='circle'),
#     hovertext=house['이름']
# )
# fig.add_trace(house_trance)



# Customize the map layout
map_style = ["open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" ,"stamen-watercolor" ]
fig.update_layout(mapbox_style=map_style[0])  # Choose a map style
fig.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},  # Adjust margin
    mapbox=dict(center=dict(lat=36.83682, lon=127.2077)),  # Set initial center
)

fig.show()

# 로드뷰를 통해 선별한 정류장과 별도로 고려한 지하철역 주변의 버스 정류장을 고려한 데이터 셋 만들기

In [52]:
we_selected = data_cp.copy()
we_selected = we_selected[['정류장번호','정류장명','위도','경도']]
drop_list = ['차암1통','STS','백석푸르지오아파트','라이프아파트','천안서초등학교','주공5단지아파트','만수사','충무병원']
we_selected = we_selected[~we_selected['정류장명'].isin(drop_list)]
we_selected


,정류장번호,정류장명,위도,경도
903,CAB285000993,브라운스톤아파트,36.830031,127.114282
904,CAB285000995,벽산블루밍아파트,36.826201,127.113307
125,ASB285002133,원랜드패션몰,36.823820,127.106290
2049,CAB285071547,백석아이파크2차아파트,36.824479,127.118505
877,CAB285000960,직산대림아파트,36.859895,127.120564
142,ASB285071769,시청앞사거리,36.815950,127.110030
918,CAB285001018,백석11단지,36.820165,127.117591
817,CAB285000896,쌍용패션거리,36.802048,127.131706
800,CAB285000876,우성VIP 아파트,36.798424,127.156239
864,CAB285000944,성정선경아파트,36.821308,127.136396


In [61]:
# 지하철역 주변 추가로 고려할 버스 정류장 
add_list=['두정역','천안역','다가동신성아파트','쌍용역(종합터미널방면)','불당동문화카페거리']

add_data = final_data.loc[final_data['정류장명'].isin(add_list),['정류장번호','정류장명','위도','경도']]
add_data

we_selected_add_Around_subway = pd.concat([we_selected,add_data],axis=0)
we_selected_add_Around_subway.drop_duplicates(subset='정류장명',inplace=True)
we_selected_add_Around_subway.reset_index(drop=True,inplace=True)

# final 데이터를 통해, '대분류' 칼럼을 얻습니다.
we_selected_add_Around_subway = pd.merge(we_selected_add_Around_subway,final_data[['정류장번호','대분류']],on='정류장번호',how='left')
we_selected_add_Around_subway

,정류장번호,정류장명,위도,경도,대분류
0,CAB285000993,브라운스톤아파트,36.830031,127.114282,백석동
1,CAB285000995,벽산블루밍아파트,36.826201,127.113307,백석동
2,ASB285002133,원랜드패션몰,36.823820,127.106290,백석동
3,CAB285071547,백석아이파크2차아파트,36.824479,127.118505,백석동
4,CAB285000960,직산대림아파트,36.859895,127.120564,부성동
5,ASB285071769,시청앞사거리,36.815950,127.110030,불당동
6,CAB285001018,백석11단지,36.820165,127.117591,백석동
7,CAB285000896,쌍용패션거리,36.802048,127.131706,쌍용동
8,CAB285000876,우성VIP 아파트,36.798424,127.156239,청룡동
9,CAB285000944,성정선경아파트,36.821308,127.136396,성정동


In [ ]:
# we_selected_add_Around_subway.to_csv(r'E:\제출용데이터dir\임시폴더\로드뷰필터링과필수지하철역주변데이터.csv',encoding='cp949',index=False)

# 최종 시각화

In [63]:
# 구글 맵의 좌표를 통한 지하철 데이터 셋 만들기
subway_df = pd.DataFrame(columns=['정류장명','위도','경도'])

subway_name ={'정류장명':'두정역','위도':36.833751,'경도':127.148908}
subway_df = subway_df.append(subway_name,ignore_index=True)

subway_name ={'정류장명':'천안역','위도':36.81040,'경도':127.146673}
subway_df = subway_df.append(subway_name,ignore_index=True)

subway_name ={'정류장명':'봉명역','위도':36.801467,'경도':127.136223}
subway_df = subway_df.append(subway_name,ignore_index=True)

subway_name ={'정류장명':'쌍용역','위도':36.792933,'경도':127.120126}
subway_df = subway_df.append(subway_name,ignore_index=True)

subway_df

,정류장명,위도,경도
0,두정역,36.833751,127.148908
1,천안역,36.810400,127.146673
2,봉명역,36.801467,127.136223
3,쌍용역,36.792933,127.120126


In [71]:
rand_mark_df = pd.DataFrame(columns=['이름','위도','경도'])

rand_mark ={'이름':'천안시청','위도':36.815141,'경도':127.113913}
rand_mark_df = rand_mark_df.append(rand_mark,ignore_index=True)

rand_mark_df

,이름,위도,경도
0,천안시청,36.815141,127.113913


In [78]:

# 전체 버스 정류장의 위치 시각화
fig = px.scatter_mapbox(
    scaled_data,
    lat='위도',
    lon='경도',
    color='blue',  # Set marker color to blue
    size_max=5,      # Adjust point size
    zoom=9,           # Initial map zoom level
    hover_name='정류장명',
    size='for_marker_size'
)

# # Create larger red square markers for '위도' and '경도' from the 'park' DataFrame
# park_trace = go.Scattermapbox(
#     lat=park['위도'],
#     lon=park['경도'],
#     mode='markers',
#     marker=dict(size=5, color='red', symbol='circle'),  # Increase the size to 15
#     hovertext=park['공원명']
# )
# fig.add_trace(park_trace)

# ground_truth_trace = go.Scattermapbox(
#     lat=ground_truth['위도'],
#     lon=ground_truth['경도'],
#     mode='markers',
#     marker=dict(size=30, color='yellowgreen', symbol='circle'),
#     hovertext=ground_truth['장소명']  # Replace with the appropriate column from ground_truth
# )
# fig.add_trace(ground_truth_trace)

subway_trance = go.Scattermapbox(
    lat = subway_df['위도'],
    lon = subway_df['경도'],
    mode='markers',
    marker = dict(size=6,color='black',symbol='circle'),
    hovertext=subway_df['정류장명']
)
fig.add_trace(subway_trance)


# 따릉이 설치 장소로 고려할 장소 시각화
data_cp_trance = go.Scattermapbox(
    lat = we_selected_add_Around_subway['위도'],
    lon = we_selected_add_Around_subway['경도'],
    mode='markers',
    marker = dict(size=15,color='red',symbol='circle'),
    hovertext=data_cp['정류장명']
)
fig.add_trace(data_cp_trance)

# 천안시의 랜드 마크 시각화
rand_mark_df_trance = go.Scattermapbox(
    lat = rand_mark_df['위도'],
    lon = rand_mark_df['경도'],
    mode='markers',
    marker = dict(size=6,color='green',symbol='circle'),
    hovertext=rand_mark_df['이름']
)
fig.add_trace(rand_mark_df_trance)


# # 제조업체 회사 시각화
# cmp_make_trance = go.Scattermapbox(
#     lat = cmp_make['위도'],
#     lon = cmp_make['경도'],
#     mode='markers',
#     marker = dict(size=10,color='orange',symbol='circle'),
#     hovertext=cmp_make['이름']
# )
# fig.add_trace(cmp_make_trance)

# # 생산업체 회사 시각화
# cmp_facil_trance = go.Scattermapbox(
#     lat = cmp_facil['위도'],
#     lon = cmp_facil['경도'],
#     mode='markers',
#     marker = dict(size=10,color='purple',symbol='circle'),
#     hovertext=cmp_facil['이름']
# )
# fig.add_trace(cmp_facil_trance)

# # 거주지 위치 시각화
# house_trance = go.Scattermapbox(
#     lat = house['위도'],
#     lon = house['경도'],
#     mode='markers',
#     marker = dict(size=10,color='indigo',symbol='circle'),
#     hovertext=house['이름']
# )
# fig.add_trace(house_trance)



# Customize the map layout
map_style = ["open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" ,"stamen-watercolor" ]
fig.update_layout(mapbox_style=map_style[0])  # Choose a map style
fig.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},  # Adjust margin
    mapbox=dict(center=dict(lat=36.83682, lon=127.2077)),  # Set initial center
)

fig.show()

In [79]:
import os
from datetime import datetime
import plotly.io as pio 
# Replace characters that are not allowed in filenames

current_datetime = datetime.now()
when = current_datetime.strftime('%Y-%m-%d_%H-%M-%S')
when_sanitized = when.replace(':', '-')

path = os.path.join(r'E:\제출용데이터dir\plotly시각화', f'{when_sanitized}.html')

# win_notebook
# path = os.path.join(r'C:\Users\USER\Desktop\plotly시각화',f'{when_sanitized}.html')
pio.write_html(fig, path)


# 9. 지하철주변 탐색하기

추가로 고려할, 지하철역 주변의 설치 장소를 고려하기 위해

두정역, 천안역, 봉명역, 쌍용역 주변에 공공 자전거 설치 장소로 적합한 버스 정류장을 찾습니다.

In [101]:
# 구글 맵의 좌표를 통한 지하철 데이터 셋 만들기
subway_df = pd.DataFrame(columns=['정류장명','위도','경도'])

subway_name ={'정류장명':'두정역','위도':36.833751,'경도':127.148908}
subway_df = subway_df.append(subway_name,ignore_index=True)

subway_name ={'정류장명':'천안역','위도':36.81040,'경도':127.146673}
subway_df = subway_df.append(subway_name,ignore_index=True)

subway_name ={'정류장명':'봉명역','위도':36.801467,'경도':127.136223}
subway_df = subway_df.append(subway_name,ignore_index=True)

subway_name ={'정류장명':'쌍용역','위도':36.792933,'경도':127.120126}
subway_df = subway_df.append(subway_name,ignore_index=True)

subway_df

,정류장명,위도,경도
0,두정역,36.833751,127.148908
1,천안역,36.810400,127.146673
2,봉명역,36.801467,127.136223
3,쌍용역,36.792933,127.120126


In [142]:
subway_visual = pd.read_pickle('./최종사용데이터_25개구.pickle')

# ToDo. "최종사용데이터" 파일을 통해 스케일링항 코드 추가하기


data = subway_visual.copy()

# 8개의 지역만 있는 데이터를 스케일링함


# 추가 검토 필요
##category_sum 스케일링
data['minmax_category']=(data.category_sum-min(data.category_sum))/max(data.category_sum)-min(data.category_sum)

##공원거리 스케일링
data['minmax_공원']=((data.공원거리구분_A+data.공원거리구분_B)-min(data.공원거리구분_A+data.공원거리구분_B))/max(data.공원거리구분_A+data.공원거리구분_B)-min(data.공원거리구분_A+data.공원거리구분_B)

##하천거리 스케일링
data['minmax_하천']=((data.하천거리구분_A+data.하천거리구분_B)-min(data.하천거리구분_A+data.하천거리구분_B))/max(data.하천거리구분_A+data.하천거리구분_B)-min(data.하천거리구분_A+data.하천거리구분_B)

##주거회사거리 스케일링
data['minmax_주거회사']=((data.CH거리구분_A+data.CH거리구분_B)-min(data.CH거리구분_A+data.CH거리구분_B))/max(data.CH거리구분_A+data.CH거리구분_B)-min(data.CH거리구분_A+data.CH거리구분_B)

##minmax 스케일링 후 score 합 
data['minmax_sum_score']=data.minmax_category+data.minmax_공원+data.minmax_하천+data.minmax_주거회사



data['score7']=(data.minmax_category)+data.minmax_공원+data.minmax_하천+(data.minmax_주거회사)*7
data.head(2)


# data.to_csv('최종사용데이터_25개구_스케일링.csv')
# data.to_pickle('최종사용데이터_25개구_스케일링.pickle')


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,하천거리구분_B,score,읍/면/동,대분류,minmax_category,minmax_공원,minmax_하천,minmax_주거회사,minmax_sum_score,score7
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,54,성환읍,성환읍,0.014155,0.0,0.25,0.053004,0.317158,0.635180
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,1,50,성환읍,성환읍,0.009992,0.0,0.25,0.045936,0.305928,0.581546


In [108]:
data['blue'] ='blue'
data['for_marker_size'] =5

fig = px.scatter_mapbox(
    data,
    lat='위도',
    lon='경도',
    color='score7',  # Set marker color to blue
    size_max=10,      # Adjust point size
    zoom=9,           # Initial map zoom level
    hover_name='정류장명',
    size='for_marker_size'
)



# subway_df

subway_trace = go.Scattermapbox(
    lat = subway_df['위도'],
    lon = subway_df['경도'],
    mode='markers',
    marker = dict(size=30,color='red',symbol='circle'),
    hovertext=subway_df['정류장명']
)
fig.add_trace(subway_trace)



# Customize the map layout
map_style = ["open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" ,"stamen-watercolor" ]
fig.update_layout(mapbox_style=map_style[0])  # Choose a map style
fig.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},  # Adjust margin
    mapbox=dict(center=dict(lat=36.83682, lon=127.2077)),  # Set initial center
)

fig.show()

In [109]:
import os
from datetime import datetime
import plotly.io as pio 
# Replace characters that are not allowed in filenames

current_datetime = datetime.now()
when = current_datetime.strftime('%Y-%m-%d_%H-%M-%S')
when_sanitized = when.replace(':', '-')

# path = os.path.join(r'E:\제출용데이터dir\plotly시각화', f'{when_sanitized}.html')

# win_notebook
path = os.path.join(r'C:\Users\USER\Desktop\plotly시각화',f'{when_sanitized}.html')
pio.write_html(fig, path)
